Videos & data used in Moll et al., 2025 (https://www.cell.com/current-biology/fulltext/S0960-9822(25)01100-5) for one behavioural session with Crow 1.

Currently not publically available, in the future we may share this.

In [ ]:
%load_ext autoreload
%autoreload 2

import glob
import os
import warnings
import numpy as np
from pathlib import Path
import pandas as pd

from movement.io import load_poses
from movement.kinematics import compute_pairwise_distances, compute_velocity, compute_acceleration
from movement.utils.vector import compute_norm

from ethograph.utils.paths import extract_trial_info_from_filename
from ethograph import TrialTree, add_angle_rgb_to_ds,  add_changepoints_to_ds, set_media_attrs
from ethograph.features.mov_features import compute_distance_to_constant, Position3DCalibration
from ethograph.features.changepoints import find_troughs_binary, find_nearest_turning_points_binary
from ethograph.features.preprocessing import gaussian_smoothing
warnings.filterwarnings(
    'ignore', 
    message='Confidence array was not provided.Setting to an array of NaNs',
    module='movement.validators.datasets'
)


fps = 200
good_s3d_feats = [ 326, 327, 292, 363, 219, 192, 260, 66, 332, 199,
       288, 763, 837, 182, 24, 218, 213, 21, 733, 242] 
behav_folder = r"D:\Akseli\AI_data\rawdata\sub-04_id-Crow1\ses-000_date-20241218_01\behav"
deriv_behav_folder = behav_folder.replace("rawdata", "derivatives")
dlc_3d_paths = np.sort(glob.glob(os.path.join(behav_folder, "dlc", "*_3D.csv")))



disp_xyz = [-10.23, -5.907, -1.395]
box1 = np.array([-7.08514973,  0.14055037,  0.58930513]) # front left
box2 = np.array([-6.97786264,  9.87752058,  0.96104736]) # back left
box3 = np.array([6.81007923, 9.72017014, 0.77115876]) # back right
box4 = np.array([ 6.77518696, -0.02424044,  0.60184316]) # front right


calibration = Position3DCalibration()
smoothing_params = {"sigma": 1.5, "axis": 0, "mode": "constant", "cval": np.nan}
clip_distance = 50 # cm (no distance feature exceeds this)






ds_list = []
for dlc_3d_path in dlc_3d_paths:
    ds = load_poses.from_dlc_file(dlc_3d_path, fps=fps)

    
    session_date, trial_num, bird = extract_trial_info_from_filename(dlc_3d_path)
    ds = ds.assign_coords(individuals=[bird])
    ds.attrs["trial"] = trial_num # can be number or string, has to be unique per session
    ds.attrs["session"] = f"{session_date}_{str(Path(behav_folder).parent)[-2:]}"  # e.g. "20250527_sub-03_id-Freddy"
    ds.attrs["bird"] = bird    
    

    base_name = os.path.basename(dlc_3d_path)
    # dlc_suffix = "DLC_resnet50_Felix_cross_SessionsAug1shuffle1_200000_filtered"
    
    ds = set_media_attrs(
        ds,
        cameras=[
            base_name.replace("_DLC_3D.csv", "-cam-1.mp4")
            # base_name.replace("_DLC_3D.csv", "-cam-2.mp4"),
        ],
        pose=[
            base_name.replace("_DLC_3D.csv", f"-cam-1DLC.csv"),
            base_name.replace("_DLC_3D.csv", f"-cam-2DLC.csv"),
        ],
    )
    ds.attrs["trial"] = trial_num

    # Transform 2D pose -> 3D pose
    ds = calibration.transform(ds)
    
     
    ds["position"] = gaussian_smoothing(ds.position, **smoothing_params)
    

    
    ds["velocity"] = compute_velocity(ds.position.sel(keypoints=['stickTip', 'beakTip'])).clip(min=-150, max=150)
    ds["speed"] = compute_norm(ds.velocity.sel(keypoints=['stickTip', 'beakTip']))
    
    
    smoothing_params_doubled = smoothing_params.copy()
    smoothing_params_doubled["sigma"] *= 2
    position_very_smooth = gaussian_smoothing(ds.position, **smoothing_params_doubled)
    position_very_smooth = gaussian_smoothing(ds.position, **smoothing_params_doubled)
    ds["acceleration"] = compute_acceleration(position_very_smooth.sel(keypoints=['stickTip', 'beakTip'])).clip(min=-1500, max=1500)
    
    

    # Distances
    ds["pellet_beakTip_dist"] = compute_pairwise_distances(ds.position, "keypoints", {"pellet": "beakTip"}).clip(0, clip_distance)
    ds["pellet_stickTip_dist"] = compute_pairwise_distances(ds.position, "keypoints", {"pellet": "stickTip"}).clip(0, clip_distance)
    ds["disp_beakTip_dist"] = compute_distance_to_constant(ds.position, reference_point=disp_xyz, keypoint="beakTip").clip(0, clip_distance)
    ds["disp_stickTip_dist"] = compute_distance_to_constant(ds.position, reference_point=disp_xyz, keypoint="stickTip").clip(0, clip_distance)
    
    ds["sticktip_cornerLFront_dist"] = compute_pairwise_distances(ds.position, "keypoints", {"stickTip": "box1"}).clip(0, clip_distance)
    ds["sticktip_cornerLBack_dist"] = compute_pairwise_distances(ds.position, "keypoints", {"stickTip": "box2"}).clip(0, clip_distance)
    ds["sticktip_cornerRBack_dist"] = compute_pairwise_distances(ds.position, "keypoints", {"stickTip": "box3"}).clip(0, clip_distance)
    ds["sticktip_cornerRFront_dist"] = compute_pairwise_distances(ds.position, "keypoints", {"stickTip": "box4"}).clip(0, clip_distance)

    ds["beakTip_cornerLFront_dist"] = compute_pairwise_distances(ds.position, "keypoints", {"beakTip": "box1"}).clip(0, clip_distance)
    ds["beakTip_cornerLBack_dist"] = compute_pairwise_distances(ds.position, "keypoints", {"beakTip": "box2"}).clip(0, clip_distance)
    ds["beakTip_cornerRBack_dist"] = compute_pairwise_distances(ds.position, "keypoints", {"beakTip": "box3"}).clip(0, clip_distance)
    ds["beakTip_cornerRFront_dist"] = compute_pairwise_distances(ds.position, "keypoints", {"beakTip": "box4"}).clip(0, clip_distance)


    # Only keep subset of keypoints
    ds = ds.sel(keypoints=["beakTip", "stickTip", "pellet"])


    s3d_file = base_name.replace("_DLC_3D.csv", "-cam-1_s3d.npy")
    s3d_path = os.path.join(behav_folder, "s3d", s3d_file)
    s3d_data = np.load(s3d_path)
    
    ds["s3d"] = (("time", "s3d_dims"), s3d_data[:, good_s3d_feats])


    for var in ds.data_vars:
        ds[var].attrs["type"] = "features"
        
    
    ds = add_changepoints_to_ds(
        ds=ds, 
        target_feature="speed",
        changepoint_name="troughs", 
        changepoint_func=find_troughs_binary, 
        prominence=0.5, 
        distance=2
    )
    
    ds = add_changepoints_to_ds(
        ds=ds,
        target_feature="speed",
        changepoint_name="turning_points",
        changepoint_func=find_nearest_turning_points_binary,
        threshold=1.0,
        max_value=50,
        prominence=5,
        width=2
    )
    
    ds = add_angle_rgb_to_ds(ds, smoothing_params=smoothing_params)
    

    if int(trial_num) == 41:
        ds.attrs["pellet_position"] = "right"
    if int(trial_num) == 115:
        ds.attrs["pellet_position"] = "left"


    # FOR LATER
    # # Pellet position and num pellets is detected from video using custom matlab code (not shared). Saved in 'trial_info.csv'.
    # ADD CSV
    # if trial_num in trial_info.index:
    #     row = trial_info.loc[trial_num]
    #     poscat_map = {0: "no pellet", 1: "left", 2: "middle", 3: "right"}
    #     ds.attrs["pellet_position"] = poscat_map.get(int(row["poscat"]))
    #     ds.attrs["num_pellets"] = int(row["num_pellets"])
        
    ds_list.append(ds)



dt = TrialTree.from_datasets(ds_list)
dt.to_netcdf(os.path.join(deriv_behav_folder, "Trial_data.nc"))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
<class 'int'>
<class 'int'>
Extracted type_vars_dict: {'individuals': array(['Crow1'], dtype='<U5'), 'cameras': array(['2024-12-17_115_Crow1-cam-1.mp4'], dtype='<U30'), 'pose': array(['2024-12-17_115_Crow1-cam-1DLC.csv',
       '2024-12-17_115_Crow1-cam-2DLC.csv'], dtype='<U33'), 'features': ['position', 'confidence', 'velocity', 'speed', 'acceleration', 'pellet_beakTip_dist', 'pellet_stickTip_dist', 'disp_beakTip_dist', 'disp_stickTip_dist', 'sticktip_cornerLFront_dist', 'sticktip_cornerLBack_dist', 'sticktip_cornerRBack_dist', 'sticktip_cornerRFront_dist', 'beakTip_cornerLFront_dist', 'beakTip_cornerLBack_dist', 'beakTip_cornerRBack_dist', 'beakTip_cornerRFront_dist', 's3d', 'angles'], 'colors': ['angle_rgb'], 'changepoints': ['speed_troughs', 'speed_turning_points'], 's3d_dims': array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19]), 'keypoints': array(['be

In [14]:
dt.itrial(0)

<xarray.DatasetView> Size: 723kB
Dimensions:                     (time: 1169, space: 3, keypoints: 3,
                                 individuals: 1, s3d_dims: 20, RGB: 3)
Coordinates:
  * time                        (time) float64 9kB 0.0 0.005 0.01 ... 5.835 5.84
  * space                       (space) <U1 12B 'x' 'y' 'z'
  * keypoints                   (keypoints) <U15 180B 'beakTip' ... 'pellet'
  * individuals                 (individuals) <U5 20B 'Crow1'
Dimensions without coordinates: s3d_dims, RGB
Data variables: (12/22)
    position                    (time, space, keypoints, individuals) float64 84kB ...
    confidence                  (time, keypoints, individuals) float32 14kB n...
    velocity                    (time, space, keypoints, individuals) float64 84kB ...
    speed                       (time, keypoints, individuals) float64 28kB n...
    acceleration                (time, space, keypoints, individuals) float64 84kB ...
    pellet_beakTip_dist         (time) float64 9kB nan nan nan ... nan nan nan
    ...                          ...
    beakTip_cornerRFront_dist   (time) float64 9kB nan nan nan ... nan nan nan
    s3d                         (time, s3d_dims) float64 187kB 0.0 0.0 ... 0.0
    speed_troughs               (keypoints, individuals, time) int8 4kB 0 ... 0
    speed_turning_points        (keypoints, individuals, time) int8 4kB 0 ... 0
    angles                      (keypoints, individuals, time) float64 28kB 1...
    angle_rgb                   (keypoints, individuals, time, RGB) float64 84kB ...
Attributes:
    source_software:  DeepLabCut
    ds_type:          poses
    fps:              200.0
    time_unit:        seconds
    source_file:      D:/Akseli/AI_data/rawdata/sub-04_id-Crow1/ses-000_date-...
    trial:            115
    session:          2024-12-17_01
    bird:             Crow1
    cameras:          ['2024-12-17_115_Crow1-cam-1.mp4']
    pose:             ['2024-12-17_115_Crow1-cam-1DLC.csv', '2024-12-17_115_C...
    pellet_position:  right
    num_pellets:      1